In [ ]:
# Import necessary libraries for numerical operations and visualizations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import heapq
import cv2

# Define the dimensions of the map and obstacle areas
map_width, map_height = 1200, 500

# Initialize starting and destination points
start_point = (50, 50)
end_point = (1150, 50)

# Function to ensure a point lies within the defined map boundaries
def is_within_map_bounds(x_coord, y_coord):
    return 5 <= x_coord < map_width - 5 and 5 <= y_coord < map_height - 5

# Determine if a point is inside a hexagonal obstacle given its center and radius
def point_in_hexagon(point_x, point_y, hex_center_x, hex_center_y, radius):
    q2x = np.fabs(point_x - hex_center_x)  # X distance to hexagon's center
    q2y = np.fabs(point_y - hex_center_y)  # Y distance to hexagon's center
    # Check against hexagon geometry to decide if inside
    return q2x * np.sqrt(3) / 2 + q2y / 2 <= radius and q2y <= radius


# Verify if a specific location is not obstructed by obstacles
def is_location_free(x_coord, y_coord):
    clearance = 5  # Set a clearance buffer around obstacles
    if 0 <= x_coord <= map_width and 0 <= y_coord <= clearance:
        return False
    if 0 <= x_coord <= clearance and 0 <= y_coord <= map_height:
        return False
    if map_width - clearance <= x_coord <= map_width and 0 <= y_coord <= map_height:
        return False
    if 0 <= x_coord <= map_width and map_height - clearance <= y_coord <= map_height:
        return False
    
    
    # Check against a rectangular obstacle
    if 100-clearance <= x_coord <= 170+clearance and 100-clearance <= y_coord <= 500+clearance:
        return False
    # Check another rectangular obstacle
    if 275-clearance <= x_coord <= 350+clearance and 0 <= y_coord <= 400+clearance:
        return False
    if 900 - clearance <= x_coord <= 1100+clearance  and 50 - clearance <= y_coord <= 125+clearance:  
        return False

    if 900 - clearance <= x_coord <= 1100 + clearance and 375 - clearance <= y_coord <= 450 + clearance: 
        return False

    if 1025 - clearance <= x_coord <= 1100 + clearance and 50 - clearance <= y_coord <= 450 + clearance: 
        return False
    # Check a hexagonal obstacle by invoking point_in_hexagon
    hex_center_x, hex_center_y, hex_radius = 650, 250, 100 + clearance
    if point_in_hexagon(x_coord, y_coord, hex_center_x, hex_center_y, hex_radius):
        return False
    return True

# Alternative function without clearance for comparison
def is_location_free_strict(x_coord, y_coord):
    # Rectangular obstacle check
    if 100 <= x_coord <= 170 and 100 <= y_coord <= 500:
        return False
    # Another rectangular obstacle check
    if 275 <= x_coord <= 350 and 0 <= y_coord <= 400:
        return False
    if 900 <= x_coord <= 1100  and 50 <= y_coord <= 125:  
        return False

    if 900<= x_coord <= 1100  and 375  <= y_coord <= 450 : 
        return False

    if 1025  <= x_coord <= 1100  and 50  <= y_coord <= 450 :  
        return False
    # Check a hexagonal obstacle by invoking point_in_hexagon
    hex_center_x, hex_center_y, hex_radius = 650, 250, 100
    if point_in_hexagon(x_coord, y_coord, hex_center_x, hex_center_y, hex_radius):
        return False
    # Assume location is free if it doesn't match any obstacles
    return True

# Define potential movement actions with their respective costs
def get_movement_actions():
    actions = {
        (-1, 0): 1.0,  # Up
        (1, 0): 1.0,   # Down
        (0, -1): 1.0,  # Left
        (0, 1): 1.0,   # Right
        (-1, -1): 1.4, # Up-Left Diagonal
        (-1, 1): 1.4,  # Up-Right Diagonal
        (1, -1): 1.4,  # Down-Left Diagonal
        (1, 1): 1.4    # Down-Right Diagonal
    }
    return actions

def dijkstra_search(start_point, end_point):
    open_list = []  # Priority queue for nodes to explore
    closed_list = set()  # Set to track explored nodes

    parent_track = {}  # Dictionary to map each node to its parent
    cost_track = {start_point: 0}  # Dictionary to track the cost from start node to each node
    heapq.heappush(open_list, (0, start_point))  # Initialize open list with start node and cost 0
    parent_track[start_point] = None  # Start node has no parent
    
    actions = get_movement_actions()  # Retrieve all possible movement actions and their costs
    
    search_steps = []

    while open_list:  # Continue until there are nodes to explore
        current_cost, current_loc = heapq.heappop(open_list)  # Pop the node with the lowest cost
        if current_loc in closed_list:  # Skip if the node has already been explored
            continue
        
        if current_loc == end_point:  # Check if the current node is the goal
            path = []  # Initialize the path
            while current_loc:  # Trace back the path from goal to start
                path.append(current_loc)  # Add current node to the path
                current_loc = parent_track[current_loc]  # Move to the parent node
            return path[::-1], search_steps  # Return the path in start to goal order
        
        closed_list.add(current_loc)  # Mark the current node as explored
        
        for move, cost in actions.items():  # Iterate through all possible actions
            next_loc = (current_loc[0] + move[0], current_loc[1] + move[1])  # Calculate next node position
            total_cost = current_cost + cost  # Calculate total cost to reach the next node
            # Check if next node is within bounds, free, and not yet closed
            if is_within_map_bounds(*next_loc) and is_location_free(*next_loc) and next_loc not in closed_list:
                # Check if next node is either unvisited or found a cheaper path
                if next_loc not in cost_track or total_cost < cost_track[next_loc]:
                    heapq.heappush(open_list, (total_cost, next_loc))  # Add/Update the node in the open list
                    parent_track[next_loc] = current_loc  # Update the parent of the next node
                    cost_track[next_loc] = total_cost  # Update the cost to reach the next node
                    search_steps.append(next_loc)
    
    return [], []  # Return an empty path if the goal is unreachable

# Function to visually represent the path found by Dijkstra's algorithm
def show_path_visual(path, search_steps):
    image = np.zeros((map_height, map_width, 3), dtype=np.uint8)
    cv2.namedWindow("Pathfinding Visualization", cv2.WINDOW_AUTOSIZE)
    # Mark obstacles and free spaces with different colors
    for x in range(map_width):
        for y in range(map_height):
            if not is_location_free(x, y):
                inverted_y = map_height - y - 1
                image[inverted_y, x] = [238, 247, 240]  # Obstacle color
            if not is_location_free_strict(x, y):
                inverted_y = map_height - y - 1
                image[inverted_y, x] = [162, 187, 1]  # Strict obstacle color
    
    radius = 1
    for i, node in enumerate(search_steps):
        cv2.circle(image, (node[0], map_height - node[1] - 1), radius, (126, 115, 93), -1)
        if i % 100 == 0 :  # Check if node is in free space
            cv2.imshow("Pathfinding Visualization", image)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit early
                break
    
    # Update frame for each step in the path
    for step in path:
        cv2.circle(image, (step[0], map_height - step[1]), 2, (124, 240, 255), -1)
        cv2.imshow("Pathfinding Visualization", image)
        cv2.waitKey(10)  # Wait a bit and allow exit with 'q'

    cv2.waitKey(0)
    cv2.destroyAllWindows()

path, search_steps = dijkstra_search(start_point, end_point)
if path != []:
    show_path_visual(path, search_steps)
else:
    print("Not Reachable")
